# MISSING REPORTS

**Se crea un csv por cada estacion con el número de valores faltantes, valores totales y porcentaje de valores faltantes por variable**

In [2]:
# Informe de valores faltantes por fichero
# Requisitos: pandas
# En un notebook pegue y ejecute esta celda

import pandas as pd
from pathlib import Path

# Lista de ficheros que ha proporcionado
file_paths = [
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Babel_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Pla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Rabassa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Albalat_Taronjers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alcora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Algar_Palancia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Beniganim_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Buñol_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Almassora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Burriana_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Ermita_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Grau_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Patronat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Penyeta_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Caudete_Fuentes_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Cirat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Coratxa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__El_Pinos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_agroalimentari_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_bomberos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Gandia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Morella_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Onda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Ontinyent_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Paterna_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_CEA_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Nord_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Port_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Sant_Jordi_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Torrebaja_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Av_Francia_completo.csv",    
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_B_Sur_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Burjasot_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Moli_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Politecnic_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Quart_Poblet_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Silla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Vivers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villamarxant_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villar_Arzobispo_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Viver_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Zorita_completo.csv"
]

# Variables a analizar
columns = ["datetime", "NO", "NO2", "NOx", "O3", "Veloc.", "Direc.", "Temp.", "R.Sol.", "Pres."]

# Parámetros de lectura
sep = ","
decimal = "."
parse_dates = ["datetime"]

# Carpeta de salida para los informes
output_dir = Path.cwd() / "missing_reports"
output_dir.mkdir(parents=True, exist_ok=True)

def report_missing_by_file(path_str, columns, sep=",", decimal=".", parse_dates=None, output_dir=Path(".") ):
    path = Path(path_str)
    if not path.exists():
        print(f"Fichero no encontrado: {path}")
        return None

    try:
        df = pd.read_csv(path, sep=sep, decimal=decimal, parse_dates=parse_dates, encoding="utf-8")
    except Exception:
        # Intento de lectura sin parseo de fechas si falla el primero
        df = pd.read_csv(path, sep=sep, decimal=decimal, encoding="utf-8")

    # Añadir columnas ausentes para homogeneizar el informe
    for col in columns:
        if col not in df.columns:
            df[col] = pd.NA

    total = len(df)
    # Evitar división por cero
    if total == 0:
        print(f"Fichero vacio: {path.name}")
        return None

    missing_count = df[columns].isna().sum().astype(int)
    missing_pct = (missing_count / total * 100).round(4)

    report_df = pd.DataFrame({
        "variable": columns,
        "missing_count": [int(missing_count[c]) for c in columns],
        "total_count": [int(total) for _ in columns],
        "missing_pct": [float(missing_pct[c]) for c in columns]
    })

    # Guardar informe
    report_name = f"{path.stem}_missing_report.csv"
    report_path = output_dir / report_name
    report_df.to_csv(report_path, index=False)

    # Mostrar resumen breve en pantalla
    top = report_df.sort_values("missing_pct", ascending=False).head(5)
    print(f"Informe generado: {report_path}  filas: {total}")
    print("Cinco variables con mayor porcentaje de faltantes")
    print(top.to_string(index=False))
    print("-" * 60)

    return report_path

# Ejecutar para todos los ficheros
for fp in file_paths:
    report_missing_by_file(fp, columns, sep=sep, decimal=decimal, parse_dates=parse_dates, output_dir=output_dir)

print("Proceso finalizado. Los informes individuales estan en:", output_dir)


Informe generado: /Users/benjamincarbonell/Documents/GitHub/TFGFinal/Codigo/missing_reports/df__Alacant_Babel_completo_missing_report.csv  filas: 134778
Cinco variables con mayor porcentaje de faltantes
variable  missing_count  total_count  missing_pct
   Pres.         134778       134778     100.0000
      NO          11678       134778       8.6646
     NO2          11676       134778       8.6631
     NOx          11675       134778       8.6624
  Veloc.           9138       134778       6.7800
------------------------------------------------------------
Informe generado: /Users/benjamincarbonell/Documents/GitHub/TFGFinal/Codigo/missing_reports/df__Alacant_Pla_completo_missing_report.csv  filas: 133369
Cinco variables con mayor porcentaje de faltantes
variable  missing_count  total_count  missing_pct
  R.Sol.         133369       133369        100.0
  Veloc.         133369       133369        100.0
   Pres.         133369       133369        100.0
  Direc.         133369       13336

# MISSING PERCENTAJE PER STATION

**Se crea un csv con el porcentaje de valores faltantes de cada variable por estacion**

In [3]:
import pandas as pd
from pathlib import Path

# Lista de ficheros proporcionada
file_paths = [
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Babel_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Pla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Rabassa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Albalat_Taronjers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alcora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Algar_Palancia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Beniganim_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Buñol_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Almassora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Burriana_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Ermita_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Grau_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Patronat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Penyeta_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Caudete_Fuentes_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Cirat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Coratxa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__El_Pinos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_agroalimentari_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_bomberos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Gandia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Morella_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Onda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Ontinyent_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Paterna_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_CEA_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Nord_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Port_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Sant_Jordi_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Torrebaja_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Av_Francia_completo.csv",    
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_B_Sur_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Burjasot_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Moli_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Politecnic_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Quart_Poblet_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Silla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Vivers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villamarxant_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villar_Arzobispo_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Viver_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Zorita_completo.csv"
]

# Variables que se desean en el informe
variables = ["NO", "NO2", "NOx", "O3", "Veloc.", "Direc.", "Temp.", "R.Sol.", "Pres."]

# Parámetros de lectura
sep = ","
encoding = "utf-8"
parse_dates = ["datetime"]

# Directorio y nombre del fichero resultado
output_path = Path.cwd() / "missing_percent_per_station.csv"

rows = []

for fp in file_paths:
    p = Path(fp)
    if not p.exists():
        # Si el fichero no existe, añadimos una fila con todos los valores NA para poder identificarlo luego
        row = {"Estaciones": p.stem}
        for v in variables:
            row[v] = float("nan")
        rows.append(row)
        print(f"Fichero no encontrado: {p}")
        continue

    try:
        df = pd.read_csv(p, sep=sep, encoding=encoding, parse_dates=parse_dates)
    except Exception:
        # Si falla el parseo de fechas, intentamos leer sin parsear
        df = pd.read_csv(p, sep=sep, encoding=encoding)

    total = len(df)
    # Evitar divisiones por cero en caso de fichero vacio
    if total == 0:
        row = {"Estaciones": p.stem}
        for v in variables:
            row[v] = 100.0
        rows.append(row)
        print(f"Fichero vacio: {p.name}")
        continue

    # Calculo del porcentaje faltante por variable
    row = {"Estaciones": p.stem}
    for v in variables:
        if v in df.columns:
            missing = df[v].isna().sum()
            pct = (missing / total) * 100.0
            row[v] = round(pct, 4)
        else:
            # Si la columna no existe en el fichero se considera 100 por ciento faltante
            row[v] = 100.0

    rows.append(row)

# Construir DataFrame y guardar CSV
result_df = pd.DataFrame(rows)
# Ordenar columnas para que Estaciones vaya primero
cols = ["Estaciones"] + variables
result_df = result_df[cols]
result_df.to_csv(output_path, index=False)

print("Informe generado en:", output_path)
print("Muestras iniciales del informe")
print(result_df.head(10).to_string(index=False))


Fichero no encontrado: /Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Buñol_completo.csv
Informe generado en: /Users/benjamincarbonell/Documents/GitHub/TFGFinal/Codigo/missing_percent_per_station.csv
Muestras iniciales del informe
                    Estaciones      NO     NO2     NOx      O3   Veloc.   Direc.    Temp.   R.Sol.  Pres.
    df__Alacant_Babel_completo  8.6646  8.6631  8.6624  1.1560   6.7800   2.7801   0.1647   0.4800  100.0
      df__Alacant_Pla_completo  4.9284  4.9052  4.9269  2.9647 100.0000 100.0000 100.0000 100.0000  100.0
  df__Alacant_Rabassa_completo  5.0781  5.0781  5.0810  1.2294   0.4513   2.0220   0.8910   1.4015  100.0
df__Albalat_Taronjers_completo  4.2711  4.2720  4.2728  1.7288 100.0000 100.0000 100.0000 100.0000  100.0
           df__Alcora_completo  7.7153  7.7148  7.6748  2.4528 100.0000 100.0000 100.0000 100.0000  100.0
   df__Algar_Palancia_completo  6.2546  6.2546  6.2512  2.6527 100.0000 100.

# MISSING FLAG PER STATION %

**Se crea un csv con un si o no (si'%<15', no'%>=15') de cada variable por estacion**

In [4]:
import pandas as pd
from pathlib import Path

# Lista completa de ficheros tal como usted proporcionó
file_paths = [
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Babel_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Pla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Rabassa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Albalat_Taronjers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alcora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Algar_Palancia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Beniganim_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Buñol_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Almassora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Burriana_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Ermita_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Grau_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Patronat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Penyeta_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Caudete_Fuentes_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Cirat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Coratxa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__El_Pinos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_agroalimentari_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_bomberos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Gandia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Morella_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Onda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Ontinyent_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Paterna_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_CEA_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Nord_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Port_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Sant_Jordi_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Torrebaja_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Av_Francia_completo.csv",    
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_B_Sur_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Burjasot_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Moli_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Politecnic_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Quart_Poblet_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Silla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Vivers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villamarxant_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villar_Arzobispo_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Viver_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Zorita_completo.csv"
]

# Variables objetivo
variables = ["NO", "NO2", "NOx", "O3", "Veloc.", "Direc.", "Temp.", "R.Sol.", "Pres."]

# Parámetros de lectura
sep = ","
encoding = "utf-8"
parse_dates = ["datetime"]

# Umbral en porcentaje
threshold = 20.0

# Ruta de salida
output_path = Path.cwd() / "missing_flag_per_station.csv"

rows = []

for fp in file_paths:
    p = Path(fp)
    station_name = p.stem  # por ejemplo df__Alacant_Babel_completo

    # Inicializar fila con 'no' por defecto
    row = {"Estaciones": station_name}
    for v in variables:
        row[v] = "no"

    if not p.exists():
        print(f"Fichero no encontrado: {p.name}. Se marcará 'no' en todas las variables.")
        rows.append(row)
        continue

    try:
        df = pd.read_csv(p, sep=sep, encoding=encoding, parse_dates=parse_dates)
    except Exception:
        df = pd.read_csv(p, sep=sep, encoding=encoding)

    total = len(df)
    if total == 0:
        print(f"Fichero vacío: {p.name}. Se marcará 'no' en todas las variables.")
        rows.append(row)
        continue

    for v in variables:
        if v in df.columns:
            missing = df[v].isna().sum()
            pct = (missing / total) * 100.0
            # si el porcentaje faltante es menor que el umbral escribimos "si"
            row[v] = "si" if pct < threshold else "no"
        else:
            # columna ausente => tratamos como 100 por ciento faltante
            row[v] = "no"

    rows.append(row)

# Guardar resultado
result_df = pd.DataFrame(rows)
cols = ["Estaciones"] + variables
result_df = result_df[cols]
result_df.to_csv(output_path, index=False)

print("CSV generado en:", output_path)
print("Primeras filas del resultado")
print(result_df.head(10).to_string(index=False))


Fichero no encontrado: df__Buñol_completo.csv. Se marcará 'no' en todas las variables.
CSV generado en: /Users/benjamincarbonell/Documents/GitHub/TFGFinal/Codigo/missing_flag_per_station.csv
Primeras filas del resultado
                    Estaciones NO NO2 NOx O3 Veloc. Direc. Temp. R.Sol. Pres.
    df__Alacant_Babel_completo si  si  si si     si     si    si     si    no
      df__Alacant_Pla_completo si  si  si si     no     no    no     no    no
  df__Alacant_Rabassa_completo si  si  si si     si     si    si     si    no
df__Albalat_Taronjers_completo si  si  si si     no     no    no     no    no
           df__Alcora_completo si  si  si si     no     no    no     no    no
   df__Algar_Palancia_completo si  si  si si     no     no    no     no    no
        df__Beniganim_completo no  no  no si     no     no    si     no    no
            df__Buñol_completo no  no  no no     no     no    no     no    no
      df__C_Almassora_completo si  si  si no     si     si    no     no    no


# MISSING BY TRANSECT

**Se crea un csv por transectos con el porcentaje de valores faltantes de cada variable por estacion con una columna final de si o no si una de las variables "NO", "NO2", "NOx", "O3" %<15**

In [5]:
# -*- coding: utf-8 -*-
"""
Genera un CSV agrupado por transectos con:
 - columna Transecto
 - columna Estaciones (nombre legible)
 - columnas de porcentaje faltante para: NO, NO2, NOx, O3, Veloc., Direc., Temp., R.Sol., Pres.
 - columna final Cumple_NO_NO2_NOx_O3 con "si" si todas las cuatro variables NO, NO2, NOx, O3 tienen <= 15% faltantes, sino "no".

Instrucciones:
 - Coloque esta celda en un notebook (VSCode) y ejecute.
 - Requiere pandas.
 - El resultado se guarda como 'missing_by_transect.csv' en el directorio de trabajo actual.
"""

import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------
# Lista de ficheros originales que usted proporcionó (sin cambios)
# ------------------------------------------------------------------
file_paths = [
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Babel_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Pla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alacant_Rabassa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Albalat_Taronjers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Alcora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Algar_Palancia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Beniganim_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Buñol_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Almassora_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Burriana_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Ermita_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Grau_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Patronat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__C_Penyeta_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Caudete_Fuentes_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Cirat_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Coratxa_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__El_Pinos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_agroalimentari_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Elxe_bomberos_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Gandia_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Morella_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Onda_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Ontinyent_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Paterna_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_CEA_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Nord_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__S_Port_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Sant_Jordi_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Torrebaja_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Av_Francia_completo.csv",    
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_B_Sur_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Burjasot_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Moli_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Politecnic_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Quart_Poblet_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Silla_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__V_Vivers_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villamarxant_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Villar_Arzobispo_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Viver_completo.csv",
    "/Users/benjamincarbonell/Documents/GitHub/TFG_Benjamin_Carbonell_Mi-ana/TFG/Datos TFG /df_completos/df__Zorita_completo.csv"
]

# ------------------------------------------------------------------
# Variables objetivo y parámetros
# ------------------------------------------------------------------
variables = ["NO", "NO2", "NOx", "O3", "Veloc.", "Direc.", "Temp.", "R.Sol.", "Pres."]
sep = ","
encoding = "utf-8"
parse_dates = ["datetime"]
threshold = 20.0   # umbral inclusivo: <= 15% => "si"

# ------------------------------------------------------------------
# Mapeo estación legible -> ruta de fichero
# Nota: use las rutas exactas proporcionadas anteriormente
# ------------------------------------------------------------------
path_map = { Path(p).stem: p for p in file_paths }

# Para facilitar, defino un diccionario explícito de estaciones legibles
# con la correspondencia al stem de fichero en 'path_map'.
# Si la estación no aparece en path_map, el script tratará como fichero no encontrado.
station_to_stem = {
    # Alicante (Transectos 1 y 2)
    "El Pla": "df__Alacant_Pla_completo",
    "Florida Babel": "df__Alacant_Babel_completo",
    "Rabassa": "df__Alacant_Rabassa_completo",
    "Elda": "df__Elda_completo",
    "El Pinos": "df__El_Pinos_completo",
    # Elche (dos estaciones separadas)
    "Elche_agroalimentari": "df__Elxe_agroalimentari_completo",
    "Elche_bomberos": "df__Elxe_bomberos_completo",
    # Valencia (Transecto 3)
    "Gandia": "df__Gandia_completo",
    "Beniganim": "df__Beniganim_completo",
    "Ontinyent": "df__Ontinyent_completo",
    # Transecto 4 and 5 (V_* files)
    "Av_Francia": "df__V_Av_Francia_completo",
    "Politecnic": "df__V_Politecnic_completo",
    "Viveros": "df__V_Vivers_completo",
    "Pista_Silla": "df__V_Silla_completo",
    "Bulevar_Sur": "df__V_B_Sur_completo",
    "Moli_del_Sol": "df__V_Moli_completo",
    "Burjassot": "df__V_Burjasot_completo",
    "Quart_Poblet": "df__V_Quart_Poblet_completo",
    "Buñol": "df__Buñol_completo",  # fichero no proporcionado -> tratado como no encontrado
    "Caudete_de_las_Fuentes": "df__Caudete_Fuentes_completo",
    # Transecto 5 extra
    "Paterna": "df__Paterna_completo",
    "Vilamarxant": "df__Villamarxant_completo",
    "Villar_Arzobispo": "df__Villar_Arzobispo_completo",
    "Torrebaja": "df__Torrebaja_completo",
    # Transecto 6 (Sagunt, Albalat, Algar, Viver)
    "S_CEA": "df__S_CEA_completo",
    "S_Port": "df__S_Port_completo",
    "S_Nord": "df__S_Nord_completo",
    "Albalat_dels_Tarongers": "df__Albalat_Taronjers_completo",
    "Algar_Palancia": "df__Algar_Palancia_completo",
    "Viver": "df__Viver_completo",
    # Transecto 7 & 8 (Castellón)
    "Burriana": "df__C_Burriana_completo",
    "Almassora": "df__C_Almassora_completo",
    "Ermita": "df__C_Ermita_completo",
    "Grau": "df__C_Grau_completo",           # Garu -> Grau
    "Patronat": "df__C_Patronat_completo",
    "Penyeta": "df__C_Penyeta_completo",
    "Onda": "df__Onda_completo",
    "Cirat": "df__Cirat_completo",
    "Alcora": "df__Alcora_completo",
    # Transecto 9
    "Sant_Jordi": "df__Sant_Jordi_completo",
    "Coratxa": "df__Coratxa_completo",
    "Morella": "df__Morella_completo",
    "Zorita": "df__Zorita_completo"
}

# ------------------------------------------------------------------
# Definición explícita de transectos (en el orden indicado)
# Use nombres legibles para las estaciones tal como aparecen en station_to_stem
# ------------------------------------------------------------------
transectos = {
    "Transecto 1 (Alicante)": [
        "El Pla", "Florida Babel", "Rabassa", "Elda", "El Pinos"
    ],
    "Transecto 2 (Alicante)": [
        "El Pla", "Florida Babel", "Rabassa", "Elche_agroalimentari", "Elche_bomberos", "Elda", "El Pinos"
    ],
    "Transecto 3 (Valencia)": [
        "Gandia", "Beniganim", "Ontinyent"
    ],
    "Transecto 4 (Valencia)": [
        "Av_Francia", "Politecnic", "Viveros", "Pista_Silla", "Bulevar_Sur",
        "Moli_del_Sol", "Burjassot", "Quart_Poblet", "Buñol", "Caudete_de_las_Fuentes"
    ],
    "Transecto 5 (Valencia)": [
        "Av_Francia", "Politecnic", "Viveros", "Pista_Silla", "Bulevar_Sur",
        "Moli_del_Sol", "Burjassot", "Quart_Poblet", "Paterna", "Vilamarxant",
        "Villar_Arzobispo", "Torrebaja"
    ],
    "Transecto 6 (Valencia)": [
        "S_CEA", "S_Port", "S_Nord", "Albalat_dels_Tarongers", "Algar_Palancia", "Viver"
    ],
    "Transecto 7 (Castellón)": [
        "Burriana", "Almassora", "Ermita", "Grau", "Patronat", "Penyeta", "Onda", "Cirat"
    ],
    "Transecto 8 (Castellón)": [
        "Burriana", "Almassora", "Ermita", "Grau", "Patronat", "Penyeta", "Alcora"
    ],
    "Transecto 9 (Castellón)": [
        "Sant_Jordi", "Coratxa", "Morella", "Zorita"
    ]
}

# ------------------------------------------------------------------
# Función auxiliar: recuperar ruta de fichero a partir del nombre lógico de estación
# ------------------------------------------------------------------
def get_filepath_for_station(station_key):
    """
    Devuelve la ruta completa si existe en path_map; si station_to_stem tiene None
    o no se encuentra en path_map, devuelve None.
    """
    stem = station_to_stem.get(station_key)
    if stem is None:
        return None
    return path_map.get(stem)

# ------------------------------------------------------------------
# Recorremos transectos y calculamos porcentajes
# ------------------------------------------------------------------
rows = []

for tran_name, stations in transectos.items():
    for station in stations:
        fp = get_filepath_for_station(station)
        # Nombre legible para la columna Estaciones: usar station tal cual
        row = {"Transecto": tran_name, "Estaciones": station}
        # Inicializar columnas con NaN por defecto
        for var in variables:
            row[var] = float("nan")

        # Estado por defecto: si no podemos leer el fichero, consideramos "no"
        cumple_label = "no"

        if fp is None:
            # Estación sin fichero asociado en mapping (por ejemplo Buñol)
            print(f"[AVISO] fichero no definido para estación: {station} (se tratará como no encontrado).")
            # dejamos porcentajes como NaN y label "no"
        else:
            p = Path(fp)
            if not p.exists():
                print(f"[AVISO] fichero no encontrado en disco: {p.name}. Se considerará 100% faltante.")
                # opcional: asignar 100% faltante a todas las variables
                for var in variables:
                    row[var] = 100.0
                cumple_label = "no"
            else:
                try:
                    df = pd.read_csv(p, sep=sep, encoding=encoding, parse_dates=parse_dates)
                except Exception:
                    # Si falla parseo fechas, leemos sin parsear
                    df = pd.read_csv(p, sep=sep, encoding=encoding)
                total = len(df)
                if total == 0:
                    print(f"[AVISO] fichero vacío: {p.name}. Se marcará 100% faltante.")
                    for var in variables:
                        row[var] = 100.0
                    cumple_label = "no"
                else:
                    # Calcular porcentajes
                    for var in variables:
                        if var in df.columns:
                            missing = df[var].isna().sum()
                            pct = (missing / total) * 100.0
                            row[var] = round(pct, 4)
                        else:
                            # columna ausente -> considerar 100% faltante
                            row[var] = 100.0

                    # Determinar la etiqueta final: "si" si NO, NO2, NOx, O3 todos <= threshold
                    cond_vars = ["NO", "NO2", "NOx", "O3"]
                    # Extraer porcentajes calculados (si hubo NaN se interpreta como > threshold)
                    try:
                        cond_values = [float(row[v]) for v in cond_vars]
                        cumple_label = "si" if all(v <= threshold for v in cond_values) else "no"
                    except Exception:
                        cumple_label = "no"

        row["Cumple_NO_NO2_NOx_O3"] = cumple_label
        rows.append(row)

# ------------------------------------------------------------------
# Construir DataFrame y guardar CSV
# ------------------------------------------------------------------
output_path = Path.cwd() / "missing_by_transect.csv"
df_result = pd.DataFrame(rows)

# Orden columnas: Transecto, Estaciones, variables..., Cumple...
cols = ["Transecto", "Estaciones"] + variables + ["Cumple_NO_NO2_NOx_O3"]
df_result = df_result[cols]

df_result.to_csv(output_path, index=False)

print("\nCSV generado en:", output_path)
print("Muestras iniciales:")
print(df_result.head(20).to_string(index=False))


[AVISO] fichero no encontrado en disco: df__Buñol_completo.csv. Se considerará 100% faltante.

CSV generado en: /Users/benjamincarbonell/Documents/GitHub/TFGFinal/Codigo/missing_by_transect.csv
Muestras iniciales:
             Transecto           Estaciones      NO     NO2     NOx     O3   Veloc.   Direc.    Temp.   R.Sol.  Pres. Cumple_NO_NO2_NOx_O3
Transecto 1 (Alicante)               El Pla  4.9284  4.9052  4.9269 2.9647 100.0000 100.0000 100.0000 100.0000  100.0                   si
Transecto 1 (Alicante)        Florida Babel  8.6646  8.6631  8.6624 1.1560   6.7800   2.7801   0.1647   0.4800  100.0                   si
Transecto 1 (Alicante)              Rabassa  5.0781  5.0781  5.0810 1.2294   0.4513   2.0220   0.8910   1.4015  100.0                   si
Transecto 1 (Alicante)                 Elda  5.5490  5.5490  5.5490 1.4024 100.0000 100.0000  24.1072 100.0000  100.0                   si
Transecto 1 (Alicante)             El Pinos 59.3479 59.3479 59.3479 2.2238  57.3935  56.241